In [80]:
%load_ext autoreload
%autoreload 2

In [13]:
import pandas as pd
import numpy as np

## Preparation

### Package loading

(To do once per environment)

Clone the package from https://github.com/ClemenceK/bpifrance_deeptech_analysis
(You probably have already done it if you have this notebook!)

In a terminal, move to this project's folder (one step above this notebook, where the setup.py is) in your terminal 
and install from there: 

`pip install .`

Clone in the same place as the first package the package from: https://github.com/ClemenceK/deep4deep

In the terminal, stay in the same folder as before and install the deep4deep package:
`pip install ../../deep4deep/`

In [1]:
#this can be used to test if the import worked
from deep4deep.utils import simple_time_tracker

@simple_time_tracker
def test():
    print("It works if below is printed something like: test 0.0, or another figure")
    
test()

It works if below is printed something like: test 0.0, or another figure
test 0.0


### Local files preparation

In the bpideep folder, create a .env file, and in this file write:

DEALROOMAPIKEY= 'your_key' (replace by your key)

(this is to avoid loading your key on github: .env is mentionned in .gitignore to not be uploaded)

In bpifrance_deeptech_analysis, create a data and raw_data local folders, with inside:
[TODO if needed]

## Building the base data from dealroom

### Get data from Dealroom with the functions written by the former Wagon team, either by : 

- **Dealroom ID** : 
    + List all the companies Dealroom ID you want to analyse in 3 different csv according to the companies classification (deeptech, non_deeptech, almost_deeptech) and save these three csv in the folder "data". 
    + Import and use the function `getdata.getfulldata()` (former wagon team function) to get the new companies data from Dealroom and save the csv in the local folder `bpideep/rawdata`

- **Name** : use the function `company_search` on each individual company:


```
from bpideep.getdata import company_search
company_search("verkor")
```


⚠️⚠️⚠️ 
**Important note: the search for new companies would need to be performed by batch if you have a limited number of Dealroom API calls (or if you are billed according to the number of calls)**

In [2]:
from bpideep.getdata import company_search

**Code to get Dealroom data for our 9 companies:**

Note: to minimize API calls, run the code below only if you need to get new data and save the new_data. 

If you are just looking, copy demo_data.csv from the Google Drive>to copy in raw_data to your bpideep/rawdata folder (to create if needed – we don't synchronize data on github because it can be heavy) then use the code in the cell below to load it. You can also try it on just 1 or 2 companies to limit API calls.

new = ["verkor", "angell", "carbios","mastergrid","pasqal","gourmey", "Epigene Labs","SpaceSense","Kraaft"]
df = pd.DataFrame()

for company in new :
    
    tmp = company_search(company)
    df = pd.concat([df, tmp], ignore_index=True)

df.to_csv("../bpideep/rawdata/demo_data.csv", index=False)
df.head(2)

In [105]:
# code to load demo_data.csv if you are just looking for a demo (after copying it from Google Drive)
# run it anyway if you used the code in the previous cell to save data (you can change the file name in both cells)
# as the 'load_json_field' function below is made to work on a loaded csv
df = pd.read_csv("../bpideep/rawdata/demo_data.csv")

### Select the needed columns to make the data analysis easier

In [106]:
new_data = df[["id", "name", "total_funding_source", "employees",
               "employees_latest", "launch_year", "growth_stage", 
               "linkedin_url", "industries", "investors"]].copy()
new_data.head(2)

,id,name,total_funding_source,employees,employees_latest,launch_year,growth_stage,linkedin_url,industries,investors
0,1985985,Verkor,0,2-10,9.0,2020,seed,https://www.linkedin.com/company/verkor/,"[{'id': 100023, 'name': 'energy'}]","{'items': [{'id': 869605, 'name': 'EIT InnoEne..."
1,1841152,Angell,10000000,11-50,25.0,2018,early growth,https://www.linkedin.com/company/angell,"[{'id': 100111, 'name': 'transportation'}]","{'items': [{'id': 1476722, 'name': 'Groupe SEB..."


### Add nb_patents to the new_data. 

Note: In our example, as we don't have an extract from Google Patents Search, we will only create a column "nb_patents" with 0 in it.

In [107]:
new_data["nb_patents"] = np.full([new_data.shape[0], 1], 0)

### Create a new column "age" to get the age of the company thanks to the column "launch_year"

In [108]:
new_data["age"] = 2020 - new_data.launch_year

### Create a new feature "investors_type"

In [109]:
# loading a few functions to help create investors_type column
from bpideep.GetCleanData import load_json_field, get_health, investors_type, simple_fund_investors

In [110]:
# from string back to json for fields that have been "stringified" by saving to csv
new_data["investors"] = new_data["investors"].apply(load_json_field)

#extracting the types of investors from the json
new_data["investors_type"] = new_data["investors"].map(investors_type)

#encoding as 0 or 1
new_data.loc[:,'investors_type'] = new_data['investors_type'].map(simple_fund_investors)

new_data.head(2)

,id,name,total_funding_source,employees,employees_latest,launch_year,growth_stage,linkedin_url,industries,investors,nb_patents,age,investors_type
0,1985985,Verkor,0,2-10,9.0,2020,seed,https://www.linkedin.com/company/verkor/,"[{'id': 100023, 'name': 'energy'}]","{'items': [{'id': 869605, 'name': 'EIT InnoEne...",0,0,1
1,1841152,Angell,10000000,11-50,25.0,2018,early growth,https://www.linkedin.com/company/angell,"[{'id': 100111, 'name': 'transportation'}]","{'items': [{'id': 1476722, 'name': 'Groupe SEB...",0,2,0


Note: if you get mistakes, it might be because you ran the same cell multiple times, so the columns are not as expected anymore. Rerun from 2.1

### Create a new feature "health_industry"

In [115]:
new_data["health_industry"] = get_health(new_data["industries"])
new_data.head(2)

,id,name,total_funding_source,employees,employees_latest,launch_year,growth_stage,linkedin_url,industries,investors,nb_patents,age,investors_type,health_industry
0,1985985,Verkor,0,2-10,9.0,2020,seed,https://www.linkedin.com/company/verkor/,"[{'id': 100023, 'name': 'energy'}]","{'items': [{'id': 869605, 'name': 'EIT InnoEne...",0,0,1,0
1,1841152,Angell,10000000,11-50,25.0,2018,early growth,https://www.linkedin.com/company/angell,"[{'id': 100111, 'name': 'transportation'}]","{'items': [{'id': 1476722, 'name': 'Groupe SEB...",0,2,0,0


### To use the pipeline, name of the feature columns should be the same

In [117]:
new_data.rename(columns={"employees_latest": "employees_clean", "launch_year": "launch_year_clean", "growth_stage": "growth_stage_imputed"}, inplace=True)
new_data.head(2)

,id,name,total_funding_source,employees,employees_clean,launch_year_clean,growth_stage_imputed,linkedin_url,industries,investors,nb_patents,age,investors_type,health_industry
0,1985985,Verkor,0,2-10,9.0,2020,seed,https://www.linkedin.com/company/verkor/,"[{'id': 100023, 'name': 'energy'}]","{'items': [{'id': 869605, 'name': 'EIT InnoEne...",0,0,1,0
1,1841152,Angell,10000000,11-50,25.0,2018,early growth,https://www.linkedin.com/company/angell,"[{'id': 100111, 'name': 'transportation'}]","{'items': [{'id': 1476722, 'name': 'Groupe SEB...",0,2,0,0


## Scraping and adding the LinkedIn data

### Generate scripts for webscraper and scrape LinkedIn
Use the Florent_demo notebook (in notebooks folder and/or documentation floder) to scrape files from Linked In
(request a demo to Florent Martin if needed!)

The scraped data should be included in a folder `bpi_deep/scraping_data/companies_people/`
prior to calling the function "build employee_df"

### Process the scraped data to a dataframe

Create a `result_files` directory inside the `bpi_deep/scraping_data/` folder.

In [128]:
from bpideep.process_scraped_data import build_employee_df, process_employee_data

In [139]:
df_employees= process_employee_data(build_employee_df())

In [142]:
#Example of the content of the employee dataframe after processing
df_employees.head(5)

,employee_name,title,profile-href,linkedin_url,technical,founder,phd
0,Agnès Mathé,responsable communication,https://www.linkedin.com/in/agn%C3%A8s-math%C3...,https://www.linkedin.com/company/carbios,0,0,0
1,Loic Zangara,vice-president france & operations,https://www.linkedin.com/in/loic-zangara-b8190...,https://www.linkedin.com/company/mastergrid,0,0,0
2,Gilles Stedile,superviseur chantier,https://www.linkedin.com/in/gilles-stedile-28b...,https://www.linkedin.com/company/mastergrid,0,0,0
3,NaN,directeur technique,NaN,https://www.linkedin.com/company/mastergrid,1,0,0
4,Meryl Merloz,purchaser,https://www.linkedin.com/in/merylmerloz/,https://www.linkedin.com/company/mastergrid,0,0,0


In [ ]:
############################
# stopping here because i don't understand how you can scrape only now…
############################

At this point, generate scripts for profile scraping on LinkedIn and scrape

In [ ]:
# Prior to calling the function "open_founder_profile_files", the csv containing the scrapped data from founders
# should be included in a folder 'bpi_deep/scraping_data/founders_files/'
from bpideep.process_scraped_data import open_founder_profile_files, inline_profile
from bpideep.process_scraped_data import build_founders_dataframe, generate_founders_features
df_founders_raw = open_founder_profile_files()

In [ ]:
#The function "build_founders_dataframe" processes the raw df and returns a df with one line per founder
#The function "generate_founders_features" generates the new relevant features such as "founder_has_phd" etc..
df_founders = generate_founders_features(build_founders_dataframe(df_founders_raw))

In [ ]:
df_founders.head(5)

In [ ]:
# Finally, we merge founders to the full employee DF, update the feature ("technical"), and aggregate into companies
from bpideep.process_scraped_data import companies_technical_stats_with_founders_features, update_technical
df_employees_full = update_technical(df_employees, df_founders)
df_companies_stats_with_founders_features = companies_technical_stats_with_founders_features(df_employees_full)
df_companies_stats_with_founders_features.head(5)

In [ ]:
# Last optional step: merge the DF with new company features with the dealroom df (partially preprocessed above)
import pandas as pd
from bpideep.process_scraped_data import merge_initial_companies_with_founder
# df_full = pd.read_csv('../bpideep/rawdata/demo_data.csv')
final = merge_initial_companies_with_founder(new_data, df_companies_stats_with_founders_features)

## Checking the NaNs

In [10]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    9 non-null      int64  
 1   name                  9 non-null      object 
 2   total_funding_source  9 non-null      int64  
 3   employees             9 non-null      object 
 4   employees_latest      8 non-null      float64
 5   launch_year           9 non-null      int64  
 6   growth_stage          9 non-null      object 
 7   linkedin_url          9 non-null      object 
 8   industries            9 non-null      object 
 9   investors             9 non-null      object 
 10  nb_patents            9 non-null      int64  
 11  age                   9 non-null      int64  
 12  investors_type        9 non-null      object 
 13  health_industry       9 non-null      int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 1.1+ KB


It is always best to have real data, even approximated and filled manually. 
The new function GetCleanData.get_clean_data() is built to get a Dealroom database filled with some manual imputings (LinkedIn scraping + manual info collection) for the columns needed for the model. 

NB : due to the short time we had, the growth stage imputing was included in the GetCleanData.get_clean_data(). It would be better if it is part of the pipeline so that any dataset of new observations can directly be handled by the pipeline. 

For the other cells, the pipeline will be able to fill in the missing data, using the average or most frequent data in the training set. It is allowed to have empty or NaN cells for the following fields:
 
+ number of employees (column called : "employees_latest" in Dealroom and "employees_clean" in our function)
+ age (created column called "age")
+ nb_patents

+ launch_year (column called : "launch_year" in Dealroom and "lauch_year_clean" in our function) -> if the imputing growth stage function is directly in the pipeline (not in our case so far)

If you have empty cells in any other fields, the model will throw an error, so make sure to fill them.

In our example, all 9 firms have a growth_stage indicated in Dealroom. So we can directly use the pipeline to predict their classifications.

Note: To use the function GetCleanData.get_clean_data(), don't forget to save the csv files (for the patents and LinkedIn data) in the folder "data", and replace the name of the csv if different from the name written in the function.

## Using the main model to predict whether the company is deeptech

Use the model already trained on our entire dataset (1332 observations) : 

In [19]:
import dill as pickle
 
# Load pipeline from pickle file
my_pipeline = pickle.load(open("../bpideep/bpideepmodelnew.pkl","rb"))

In [ ]:
# Predict the Labels using the reloaded Model
y_pred = my_pipeline.predict(final)  

## Scraping and using text data

The input needs to have the following fields:
[TODO]

Then use the following functions:

## Using the NLP model to predict whether the company is deeptech

## Making the two predictions "vote" for final prediction